In [1]:
!gdown 1jJonmFJuF__DqAOOIQVs8KHBWKOf0x8p
!gdown 1Z2xsHZiSij2LErEnbDPsKUDOliOWfpjl

Downloading...
From (original): https://drive.google.com/uc?id=1jJonmFJuF__DqAOOIQVs8KHBWKOf0x8p
From (redirected): https://drive.google.com/uc?id=1jJonmFJuF__DqAOOIQVs8KHBWKOf0x8p&confirm=t&uuid=a577078a-35ac-4524-bd94-951c24ec0e93
To: /content/train-balanced.csv.bz2
100% 90.1M/90.1M [00:01<00:00, 47.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Z2xsHZiSij2LErEnbDPsKUDOliOWfpjl
To: /content/test-balanced.csv.bz2
100% 22.7M/22.7M [00:00<00:00, 49.9MB/s]


In [2]:
# --- Imports ---
import os, json, numpy as np, pandas as pd, torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.utils.class_weight import compute_class_weight
from datasets import Dataset, DatasetDict
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer, DataCollatorWithPadding,
                          EarlyStoppingCallback)
import seaborn as sns
import matplotlib.pyplot as plt


# Optional LoRA
try:
    from peft import LoraConfig, get_peft_model, TaskType
    PEFT_AVAILABLE = True
except Exception:
    PEFT_AVAILABLE = False

print("Torch:", torch.__version__)

Torch: 2.9.0+cu126


In [3]:
# --- Parameters ---
out_dir   = "outputs"

model_name   = "distilroberta-base"
max_len      = 64
batch_size   = 16
epochs       = 1
lr           = 3e-5
weight_decay = 0.01
seed         = 789
val_size     = 0.2

use_lora     = True
lora_r       = 8
lora_alpha   = 16
lora_dropout = 0.1

os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(seed); np.random.seed(seed)
device = "cuda" if torch.cuda.is_available() else "cpu"
device

bf16_ok = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8

In [4]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average="macro", zero_division=0)
    acc = accuracy_score(labels, preds)
    try:
        prob1 = torch.softmax(torch.tensor(logits), dim=-1)[:, 1].numpy()
        auroc = roc_auc_score(labels, prob1)
    except Exception:
        auroc = float("nan")
    return {"acc": acc, "precision": prec, "recall": rec, "f1": f1, "auroc": auroc}

In [5]:
train_df = pd.read_csv(
    "/content/train-balanced.csv.bz2",
    compression="bz2",
    sep="\t",
    header=None
)

test_df = pd.read_csv(
    "/content/test-balanced.csv.bz2",
    compression="bz2",
    sep="\t",
    header=None
)


train_df = train_df.dropna(subset=[0, 1]).reset_index(drop=True)
test_df  = test_df.dropna(subset=[0, 1]).reset_index(drop=True)

tr_df, val_df = train_test_split(
    train_df, test_size=val_size,
    stratify=train_df[0], random_state=seed
)

def to_hfds(df):
    return Dataset.from_pandas(df[[1, 0]].rename(columns={1: "text", 0: "labels"}), preserve_index=False)

ds = DatasetDict({
    "train": to_hfds(tr_df),
    "validation": to_hfds(val_df),
    "test": to_hfds(test_df)
})
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 900533
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 225134
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 283332
    })
})

In [6]:
tok = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tok(batch["text"], truncation=True, padding="max_length", max_length=max_len)

ds_tok = ds.map(tokenize, batched=True, remove_columns=["text"])

classes = np.unique(tr_df[0])
label_map = {int(c): i for i, c in enumerate(sorted(classes))}
def remap_labels(d):
    d["labels"] = [label_map[int(x)] for x in d["labels"]]
    return d

ds_tok = ds_tok.map(remap_labels, batched=True)
ds_tok.set_format("torch")
ds_tok

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Map:   0%|          | 0/283332 [00:00<?, ? examples/s]

Map:   0%|          | 0/900533 [00:00<?, ? examples/s]

Map:   0%|          | 0/225134 [00:00<?, ? examples/s]

Map:   0%|          | 0/283332 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 900533
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 225134
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 283332
    })
})

In [7]:
classes = np.unique(tr_df[0])
cw = compute_class_weight("balanced", classes=classes, y=tr_df[0].values)

class_weights = torch.tensor([cw[list(classes).index(k)] for k in sorted(classes)], dtype=torch.float)
label_map, class_weights

({0: 0, 1: 1}, tensor([1.0000, 1.0000]))

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(classes))

if use_lora:
    if not PEFT_AVAILABLE:
        raise RuntimeError("Install `peft` first: pip install peft")
    peft_cfg = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=lora_r, lora_alpha=lora_alpha, lora_dropout=lora_dropout,
        target_modules=["query","key","value","dense"]
    )
    model = get_peft_model(model, peft_cfg)
    model.print_trainable_parameters()

model.to(device);

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,255,682 || all params: 83,375,620 || trainable%: 1.5061


In [9]:
class WeightedTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights
        if class_weights is not None:
            self.class_weights = self.class_weights.to(self.model.device)

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**{k:v for k,v in inputs.items() if k!="labels"})
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights) if self.class_weights is not None else torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [10]:
targs = TrainingArguments(
    output_dir=out_dir,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=lr,
    per_device_train_batch_size=max(24, batch_size),
    per_device_eval_batch_size=max(1, batch_size*2),
    num_train_epochs=epochs,
    max_steps=5000,
    weight_decay=weight_decay,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    logging_steps=50,
    seed=seed,
    report_to="none",
    bf16=bf16_ok,
    gradient_accumulation_steps=1,    # can increase if you need a bigger effective batch
    dataloader_num_workers=2,
)

trainer = WeightedTrainer(
    class_weights=class_weights,
    model=model,
    args=targs,
    train_dataset=ds_tok["train"],
    eval_dataset=ds_tok["validation"],
    tokenizer=tok,
    data_collator=DataCollatorWithPadding(tok),
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-1763749030.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


In [11]:
from google.colab import drive
drive.mount('/content/my_drive')  # use a new folder

output_dir = "/content/my_drive/MyDrive/cs3244/models/bert_model"

os.makedirs(os.path.dirname(output_dir), exist_ok=True)

trainer.save_model(output_dir)          # Saves model weights + config
tok.save_pretrained(output_dir)         # Saves tokenizer

print(f"Saved trained model and tokenizer to {output_dir}")


Mounted at /content/my_drive
Saved trained model and tokenizer to /content/my_drive/MyDrive/cs3244/models/bert_model


In [12]:
train_result = trainer.train()
train_result.metrics

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
val_metrics = trainer.evaluate(ds_tok["validation"])
test_metrics = trainer.evaluate(ds_tok["test"])
print("Validation:", val_metrics)
print("Test:", test_metrics)

with open(os.path.join(out_dir, "metrics.json"), "w") as f:
    json.dump({"val": val_metrics, "test": test_metrics}, f, indent=2)

In [ ]:
test_predictions = trainer.predict(ds_tok["test"])

predicted_labels = test_predictions.predictions.argmax(axis=1)
true_labels = ds_tok["test"]["labels"]

cm = confusion_matrix(true_labels, predicted_labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=sorted(label_map.keys()), yticklabels=sorted(label_map.keys()))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
test_probs = torch.softmax(torch.tensor(test_predictions.predictions), dim=-1)[:, 1].numpy()


print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=[str(k) for k in sorted(label_map.keys())]))

if len(label_map) == 2:
    fpr, tpr, thresholds = roc_curve(true_labels, test_probs)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()
else:
    print("ROC curve and AUC are primarily for binary classification.")

In [ ]:
logits = test_predictions.predictions

probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()
pred_ids = probs.argmax(axis=1)
true_ids = np.array(ds_tok["test"]["labels"])


id2label = {v: k for k, v in label_map.items()}

analysis_df = test_df.copy().reset_index(drop=True)
analysis_df = analysis_df.rename(columns={0: "true_label_raw", 1: "text"})

analysis_df["true_label"] = [id2label[i] for i in true_ids]
analysis_df["pred_label"] = [id2label[i] for i in pred_ids]

# pick which id corresponds to "sarcastic" if you know; here I assume label '1'
sarcastic_id = label_map[1]  # change if your mapping is different
analysis_df["prob_sarcastic"] = probs[:, sarcastic_id]

analysis_df


In [ ]:
high_conf_sarc = analysis_df[
    (analysis_df["pred_label"] == 1) &
    (analysis_df["true_label"] == 1) &
    (analysis_df["prob_sarcastic"] > 0.9)
][["text", "true_label", "pred_label", "prob_sarcastic"]].head(10)
high_conf_sarc


In [ ]:
high_conf_not = analysis_df[
    (analysis_df["pred_label"] == 0) &
    (analysis_df["true_label"] == 0) &
    (analysis_df["prob_sarcastic"] < 0.1)
][["text", "true_label", "pred_label", "prob_sarcastic"]].head(10)
high_conf_not


In [ ]:
false_pos = analysis_df[
    (analysis_df["true_label"] == 0) &
    (analysis_df["pred_label"] == 1)
][["text", "true_label", "pred_label", "prob_sarcastic"]].head(10)


false_pos



In [ ]:
false_neg = analysis_df[
    (analysis_df["true_label"] == 1) &
    (analysis_df["pred_label"] == 0)
][["text", "true_label", "pred_label", "prob_sarcastic"]].head(10)
false_neg


#LIME Text Explanations

In [ ]:
!pip install lime

In [ ]:
from lime.lime_text import LimeTextExplainer
import numpy as np

class_names = ["not_sarcastic", "sarcastic"]

def predict_proba(texts):
    enc = tok(texts, return_tensors="pt", truncation=True, padding=True, max_length=64).to(device)
    with torch.no_grad():
        logits = model(**enc).logits
        probs = torch.softmax(logits, dim=-1).cpu().numpy()
    return probs

explainer = LimeTextExplainer(class_names=class_names)


In [ ]:
example_text_1 = test_df.iloc[290][1]  # comment text
example_text_1


In [ ]:
exp = explainer.explain_instance(
    example_text_1,
    predict_proba,
    num_features=10
)

exp.show_in_notebook(text=True)


In [ ]:
example_text_2 = test_df.iloc[12][1]  # comment text
example_text_2

In [ ]:
exp = explainer.explain_instance(
    example_text_2,
    predict_proba,
    num_features=10
)

exp.show_in_notebook(text=True)